# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [55]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [56]:
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'sk_541d349ea9ee429782bf1e5562bf74fb'

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [57]:
symbol='AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data


{'companyName': 'Apple Inc',
 'marketcap': 2913168898360,
 'week52high': 198.57,
 'week52low': 163.63,
 'week52highSplitAdjustOnly': 199.62,
 'week52lowSplitAdjustOnly': 164.08,
 'week52change': 0.09447084253169669,
 'sharesOutstanding': 15334082000,
 'float': 0,
 'avg10Volume': 41730424,
 'avg30Volume': 62011003,
 'day200MovingAvg': 182.3,
 'day50MovingAvg': 177.64,
 'employees': 164000,
 'ttmEPS': 6.43,
 'ttmDividendRate': 0.966204324874062,
 'dividendYield': 0.005085821269997168,
 'nextDividendDate': '',
 'exDividendDate': '2024-05-10',
 'nextEarningsDate': '2024-08-01',
 'peRatio': 29.01880582892548,
 'beta': 1.0670495890183638,
 'maxChangePercent': 73.6483300589391,
 'year5ChangePercent': 3.4285724276027016,
 'year2ChangePercent': 0.2923897441479193,
 'year1ChangePercent': 0.09447084253169669,
 'ytdChangePercent': -0.008040417732047778,
 'month6ChangePercent': 0.006280913290768408,
 'month3ChangePercent': 0.043068437434286455,
 'month1ChangePercent': 0.12519530116357425,
 'day30Ch

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [58]:
data['year1ChangePercent']

0.09447084253169669

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [59]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])
    
my_columns = ['Ticker', 'companyName', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)


Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [60]:
for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        new_series = pd.Series([symbol, data[symbol]['stats']['companyName'], data[symbol]['quote']['latestPrice'], data[symbol]['stats']['year1ChangePercent'], 'N/A'], index = my_columns)
        new_row_df = new_series.to_frame().T
        final_dataframe = pd.concat([final_dataframe, new_row_df], ignore_index=True)
final_dataframe

,Ticker,companyName,Price,One-Year Price Return,Number of Shares to Buy
0,A,Agilent Technologies Inc.,148.21,0.260206,N/A
1,AAL,American Airlines Group Inc,13.44,-0.03554,N/A
2,AAP,Advance Auto Parts Inc,70.18,-0.369823,N/A
3,AAPL,Apple Inc,189.99,0.094471,N/A
4,ABBV,Abbvie Inc,155.34,0.188102,N/A
...,...,...,...,...,...
491,YUM,Yum Brands Inc.,135.86,0.08063,N/A
492,ZBH,Zimmer Biomet Holdings Inc,116.33,-0.081002,N/A
493,ZBRA,Zebra Technologies Corp.,326.24,0.235267,N/A
494,ZION,Zions Bancorporation N.A,42.52,0.584704,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [61]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,companyName,Price,One-Year Price Return,Number of Shares to Buy
0,LB,L Brands Inc,79.92,2.281287,N/A
1,NVDA,NVIDIA Corp,1139.01,1.734765,N/A
2,GPS,"Gap, Inc.",20.35,1.630259,N/A
3,NRG,NRG Energy Inc.,83.85,1.616622,N/A
4,SLG,SL Green Realty Corp.,50.17,1.513777,N/A
5,TT,Trane Technologies plc,328.31,1.055683,N/A
6,GE,GE Aerospace,168.56,1.053648,N/A
7,QCOM,"Qualcomm, Inc.",213.08,1.005568,N/A
8,WRK,WestRock Co,53.32,0.957884,N/A
9,HWM,Howmet Aerospace Inc,85.28,0.954077,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [62]:
def portfolio_input():
    global portfolio_size
    portfolio_size = 100000

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

100000


In [63]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

C:\Users\trygv\AppData\Local\Temp\ipykernel_23008\2310163137.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])


,Ticker,companyName,Price,One-Year Price Return,Number of Shares to Buy
0,LB,L Brands Inc,79.92,2.281287,24
1,NVDA,NVIDIA Corp,1139.01,1.734765,1
2,GPS,"Gap, Inc.",20.35,1.630259,96
3,NRG,NRG Energy Inc.,83.85,1.616622,23
4,SLG,SL Green Realty Corp.,50.17,1.513777,39
5,TT,Trane Technologies plc,328.31,1.055683,5
6,GE,GE Aerospace,168.56,1.053648,11
7,QCOM,"Qualcomm, Inc.",213.08,1.005568,9
8,WRK,WestRock Co,53.32,0.957884,36
9,HWM,Howmet Aerospace Inc,85.28,0.954077,22


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [64]:
hqm_columns = [
                'Ticker',
                'Company Name',
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            new_series =pd.Series([symbol, 
                data[symbol]['quote']['companyName'],   
                float(data[symbol]['quote']['latestPrice']),
                'N/A',
                float(data[symbol]['stats']['year1ChangePercent']),
                'N/A',
                float(data[symbol]['stats']['month6ChangePercent']),
                'N/A',
                float(data[symbol]['stats']['month3ChangePercent']),
                'N/A',
                float(data[symbol]['stats']['month1ChangePercent']),
                'N/A',
                'N/A'
                ], 
                index = hqm_columns)
            new_row_df = new_series.to_frame().T
            hqm_dataframe = pd.concat([hqm_dataframe, new_row_df], ignore_index=True)
        except:
            pass

  
        
hqm_dataframe

,Ticker,Company Name,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,148.21,N/A,0.260206,N/A,0.209183,N/A,0.138474,N/A,0.0938,N/A,N/A
1,AAL,American Airlines Group Inc,13.44,N/A,-0.03554,N/A,0.136289,N/A,-0.109395,N/A,-0.002882,N/A,N/A
2,AAP,Advance Auto Parts Inc,70.18,N/A,-0.369823,N/A,0.373872,N/A,0.077898,N/A,-0.074128,N/A,N/A
3,AAPL,Apple Inc,189.99,N/A,0.094471,N/A,0.006281,N/A,0.043068,N/A,0.125195,N/A,N/A
4,ABBV,Abbvie Inc,155.34,N/A,0.188102,N/A,0.150632,N/A,-0.114326,N/A,-0.016038,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,YUM,Yum Brands Inc.,135.86,N/A,0.08063,N/A,0.088721,N/A,0.009387,N/A,-0.02429,N/A,N/A
488,ZBH,Zimmer Biomet Holdings Inc,116.33,N/A,-0.081002,N/A,0.026542,N/A,-0.077084,N/A,-0.024633,N/A,N/A
489,ZBRA,Zebra Technologies Corp. - Class A,326.24,N/A,0.235267,N/A,0.412527,N/A,0.185728,N/A,0.100232,N/A,N/A
490,ZION,Zions Bancorporation N.A,42.52,N/A,0.584704,N/A,0.289436,N/A,0.134807,N/A,0.051111,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [65]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        price_returns = hqm_dataframe[f'{time_period} Price Return']
        valid_price_returns = [x for x in price_returns if pd.notna(x)]        
        price_return = hqm_dataframe.loc[row, f'{time_period} Price Return']
        if pd.notna(price_return):
            hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(valid_price_returns, price_return)/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.636179
1      0.199187
2       0.01626
3      0.392276
4      0.536585
         ...   
487    0.365854
488    0.140244
489    0.605691
490    0.910569
491    0.302846
Name: One-Year Return Percentile, Length: 492, dtype: object
0       0.70935
1      0.565041
2      0.898374
3      0.247967
4      0.601626
         ...   
487    0.424797
488    0.276423
489    0.930894
490    0.821138
491    0.138211
Name: Six-Month Return Percentile, Length: 492, dtype: object
0      0.849593
1      0.134146
2      0.672764
3      0.558943
4      0.123984
         ...   
487    0.432927
488    0.191057
489    0.908537
490    0.841463
491    0.081301
Name: Three-Month Return Percentile, Length: 492, dtype: object
0      0.892276
1      0.384146
2      0.079268
3      0.930894
4      0.317073
         ...   
487    0.270325
488     0.25813
489    0.898374
490    0.723577
491    0.823171
Name: One-Month Return Percentile, Length: 492, dtype: object


,Ticker,Company Name,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,148.21,N/A,0.260206,0.636179,0.209183,0.70935,0.138474,0.849593,0.0938,0.892276,N/A
1,AAL,American Airlines Group Inc,13.44,N/A,-0.03554,0.199187,0.136289,0.565041,-0.109395,0.134146,-0.002882,0.384146,N/A
2,AAP,Advance Auto Parts Inc,70.18,N/A,-0.369823,0.01626,0.373872,0.898374,0.077898,0.672764,-0.074128,0.079268,N/A
3,AAPL,Apple Inc,189.99,N/A,0.094471,0.392276,0.006281,0.247967,0.043068,0.558943,0.125195,0.930894,N/A
4,ABBV,Abbvie Inc,155.34,N/A,0.188102,0.536585,0.150632,0.601626,-0.114326,0.123984,-0.016038,0.317073,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,YUM,Yum Brands Inc.,135.86,N/A,0.08063,0.365854,0.088721,0.424797,0.009387,0.432927,-0.02429,0.270325,N/A
488,ZBH,Zimmer Biomet Holdings Inc,116.33,N/A,-0.081002,0.140244,0.026542,0.276423,-0.077084,0.191057,-0.024633,0.25813,N/A
489,ZBRA,Zebra Technologies Corp. - Class A,326.24,N/A,0.235267,0.605691,0.412527,0.930894,0.185728,0.908537,0.100232,0.898374,N/A
490,ZION,Zions Bancorporation N.A,42.52,N/A,0.584704,0.910569,0.289436,0.821138,0.134807,0.841463,0.051111,0.723577,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [66]:
from statistics import mean

print(len(hqm_dataframe.index))
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
(hqm_dataframe)

492


,Ticker,Company Name,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,148.21,N/A,0.260206,0.636179,0.209183,0.70935,0.138474,0.849593,0.0938,0.892276,0.77185
1,AAL,American Airlines Group Inc,13.44,N/A,-0.03554,0.199187,0.136289,0.565041,-0.109395,0.134146,-0.002882,0.384146,0.32063
2,AAP,Advance Auto Parts Inc,70.18,N/A,-0.369823,0.01626,0.373872,0.898374,0.077898,0.672764,-0.074128,0.079268,0.416667
3,AAPL,Apple Inc,189.99,N/A,0.094471,0.392276,0.006281,0.247967,0.043068,0.558943,0.125195,0.930894,0.53252
4,ABBV,Abbvie Inc,155.34,N/A,0.188102,0.536585,0.150632,0.601626,-0.114326,0.123984,-0.016038,0.317073,0.394817
...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,YUM,Yum Brands Inc.,135.86,N/A,0.08063,0.365854,0.088721,0.424797,0.009387,0.432927,-0.02429,0.270325,0.373476
488,ZBH,Zimmer Biomet Holdings Inc,116.33,N/A,-0.081002,0.140244,0.026542,0.276423,-0.077084,0.191057,-0.024633,0.25813,0.216463
489,ZBRA,Zebra Technologies Corp. - Class A,326.24,N/A,0.235267,0.605691,0.412527,0.930894,0.185728,0.908537,0.100232,0.898374,0.835874
490,ZION,Zions Bancorporation N.A,42.52,N/A,0.584704,0.910569,0.289436,0.821138,0.134807,0.841463,0.051111,0.723577,0.824187


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [67]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
print(hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)['HQM Score'][:51])
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe

330    0.993394
335    0.992378
378    0.987297
226    0.984248
317    0.968496
332    0.960366
42     0.960366
476    0.957825
269    0.953252
435    0.949187
268     0.94563
237    0.942581
278    0.938008
203     0.92378
375    0.921748
36     0.919207
259    0.914634
467    0.909045
161    0.903963
177    0.893293
192    0.893293
380    0.890244
420    0.884654
108    0.880081
76     0.878049
486    0.877541
464       0.875
28     0.872459
9      0.867886
204    0.863313
323    0.861789
249    0.858232
231    0.851118
123    0.850102
371    0.848577
276    0.846037
48     0.843496
181    0.838923
489    0.835874
173    0.831809
105    0.830285
438     0.82876
490    0.824187
54     0.822663
32     0.817073
73     0.809451
316    0.804878
433    0.802337
70     0.801829
353    0.800305
196    0.799797
Name: HQM Score, dtype: object


,Ticker,Company Name,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,148.21,N/A,0.260206,0.636179,0.209183,0.70935,0.138474,0.849593,0.0938,0.892276,0.77185
1,AAL,American Airlines Group Inc,13.44,N/A,-0.03554,0.199187,0.136289,0.565041,-0.109395,0.134146,-0.002882,0.384146,0.32063
2,AAP,Advance Auto Parts Inc,70.18,N/A,-0.369823,0.01626,0.373872,0.898374,0.077898,0.672764,-0.074128,0.079268,0.416667
3,AAPL,Apple Inc,189.99,N/A,0.094471,0.392276,0.006281,0.247967,0.043068,0.558943,0.125195,0.930894,0.53252
4,ABBV,Abbvie Inc,155.34,N/A,0.188102,0.536585,0.150632,0.601626,-0.114326,0.123984,-0.016038,0.317073,0.394817
5,ABMD,Abiomed Inc.,381.02,N/A,0.0,0.237805,0.0,0.213415,0.0,0.404472,0.0,0.420732,0.319106
6,ABT,Abbott Laboratories,101.95,N/A,0.030986,0.29065,0.022005,0.268293,-0.12508,0.105691,-0.033293,0.195122,0.214939
7,ACN,Accenture plc - Class A,297.73,N/A,0.006659,0.262195,-0.087752,0.087398,-0.200457,0.046748,-0.022791,0.278455,0.168699
8,ADBE,Adobe Inc,478.43,N/A,0.144539,0.463415,-0.232273,0.022358,-0.139478,0.085366,-0.00446,0.376016,0.236789
9,ADI,Analog Devices Inc.,233.44,N/A,0.367905,0.74187,0.289233,0.817073,0.24137,0.955285,0.151211,0.957317,0.867886


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [68]:
position_size = float(100000) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

C:\Users\trygv\AppData\Local\Temp\ipykernel_23008\434239120.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])


,Ticker,Company Name,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,148.21,13,0.260206,0.636179,0.209183,0.70935,0.138474,0.849593,0.0938,0.892276,0.77185
1,AAL,American Airlines Group Inc,13.44,145,-0.03554,0.199187,0.136289,0.565041,-0.109395,0.134146,-0.002882,0.384146,0.32063
2,AAP,Advance Auto Parts Inc,70.18,27,-0.369823,0.01626,0.373872,0.898374,0.077898,0.672764,-0.074128,0.079268,0.416667
3,AAPL,Apple Inc,189.99,10,0.094471,0.392276,0.006281,0.247967,0.043068,0.558943,0.125195,0.930894,0.53252
4,ABBV,Abbvie Inc,155.34,12,0.188102,0.536585,0.150632,0.601626,-0.114326,0.123984,-0.016038,0.317073,0.394817
5,ABMD,Abiomed Inc.,381.02,5,0.0,0.237805,0.0,0.213415,0.0,0.404472,0.0,0.420732,0.319106
6,ABT,Abbott Laboratories,101.95,19,0.030986,0.29065,0.022005,0.268293,-0.12508,0.105691,-0.033293,0.195122,0.214939
7,ACN,Accenture plc - Class A,297.73,6,0.006659,0.262195,-0.087752,0.087398,-0.200457,0.046748,-0.022791,0.278455,0.168699
8,ADBE,Adobe Inc,478.43,4,0.144539,0.463415,-0.232273,0.022358,-0.139478,0.085366,-0.00446,0.376016,0.236789
9,ADI,Analog Devices Inc.,233.44,8,0.367905,0.74187,0.289233,0.817073,0.24137,0.955285,0.151211,0.957317,0.867886


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [69]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

NameError: name 'writer' is not defined

## Saving Our Excel Output

As before, saving our Excel output is very easy: